In [6]:
import netCDF4
import pycpt
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
%matplotlib inline

In [2]:
cmap = pycpt.load.cmap_from_cptcity_url('ngdc/ETOPO1.cpt')

In [3]:
ds = netCDF4.Dataset('http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/vaklodingen/vaklodingenKB135_1110.nc')

In [4]:
times = netCDF4.num2date(ds.variables['time'][:], ds.variables['time'].units)
z = ds.variables['z'][:]


In [7]:
# TODO: morph morphologies, using an approach like: https://pdfs.semanticscholar.org/4912/7dc9a652029ba08e982b11be5735ce92e440.pdf
# using cv2 dense optical flow 

In [8]:
# scipy.interpolate.LinearNDInterpolator()
timestamps = np.array([t.timestamp() for t in times])

In [15]:
def interp_bathy(timestamps, z):
    interp = scipy.interpolate.interp1d(timestamps, z)
    def fill_interp(t):
        z_interp = interp(t)
        mask = np.isnan(z_interp)
        return np.ma.masked_invalid(z_interp)
    return fill_interp

def fill_mask_with_last(arr):
    """fill masked values with previous value"""
    # indices of previous value
    prev = np.arange(len(arr))
    # keep using the same value (don't accumulate)
    prev[arr.mask] = 0
    # next value
    prev = np.maximum.accumulate(prev)
    return arr[prev]

In [16]:
# test
fill_mask_with_last(
    np.ma.masked_array([1, 2, 3, 4, 5, 0], mask=[1, 0, 1, 1, 0, 1])
)

masked_array(data = [-- 2 2 2 5 5],
             mask = [ True False False False False False],
       fill_value = 999999)

In [26]:
z_filled = np.apply_along_axis(fill_mask_with_last, 0, z)

In [18]:
import ipywidgets

In [24]:
interp = interp_bathy(timestamps, np.moveaxis(z_filled, 0, 2).filled(np.nan))

def plot(t0_idx=(0, len(times)), t1_idx=(0, len(times))):
    fig, axes = plt.subplots(1, 3, figsize=(13, 10))
    axes[0].imshow(z_filled[t0_idx], cmap=cmap, vmin=-20, vmax=20)
    axes[1].imshow(interp(0.5 * timestamps[t0_idx] + 0.5 * timestamps[t1_idx]), cmap=cmap, vmin=-20, vmax=20)
    axes[2].imshow(z_filled[t1_idx], cmap=cmap, vmin=-20, vmax=20)

In [25]:
ipywidgets.interactive(plot)

interactive(children=(IntSlider(value=8, description='t0_idx', max=17), IntSlider(value=8, description='t1_idx', max=17), Output()), _dom_classes=('widget-interact',))

In [167]:
interp(0.5 * timestamps[t0_idx] + 0.5 * timestamps[t1_idx]).shape

(625, 500, 17)